In [1]:
import datetime as dt
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import tetrion.plots as plots
import tetrion.db
import tetrion.commands as cmd
import tetrion.options
import tetrion.clientcore
import seaborn as sns
from collections import defaultdict
import numpy as np
import warnings
warnings.simplefilter("ignore")
%matplotlib inline 
symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)
import redis
import json
import matplotlib.dates as mdates
from tabulate import tabulate
import sys
import os
sys.path.append(os.path.abspath(".."))
from Backtestingtools.trade_loader import get_fills, build_df_dict2, concat_df, TradeLoader

In [ ]:
loader = TradeLoader(strat="capital_electron_cdf")

# 單日
df = loader.load_single_day("2025-05-06")

# 區間合併
df_all = loader.load_concat("2025-03-01", "2025-03-31")

# 區間按日 dict
df_dict = loader.load_by_day("2025-04-01", "2025-05-06")

Loading daily fills for capital_electron_cdf: 100%|██████████| 36/36 [00:01<00:00, 18.00it/s]


In [12]:
df

,time,ts,instr,oflag,tid,px,alqortake,sz,theo,fwd,...,gamma,theta,vol,mult,latefill,fee,qm,pfdelta,otags,seq
0,2025-05-05 20:36:59.251993,1746448619251993,FUT_TAIFEX_CDF:202505,0,TAIFEX117#12465,924,alq,1,924.5,924.5,...,0,0,0,2000,False,0.02098,0,0,0x20000,32
1,2025-05-05 20:36:59.252639,1746448619252639,FUT_TAIFEX_QFF:202505,0,TAIFEX117#12529,925,take,-10,925.5,925.5,...,0,0,0,100,False,0.06850,0,0,0x0,33
2,2025-05-05 20:36:59.252639,1746448619252639,FUT_TAIFEX_QFF:202505,0,TAIFEX117#12529,924,take,-10,924.5,924.5,...,0,0,0,100,False,0.06848,0,0,0x0,34
3,2025-05-05 20:38:22.152951,1746448702152951,FUT_TAIFEX_CDF:202505,0,TAIFEX117#12537,923,alq,1,924.0,924.0,...,0,0,0,2000,False,0.02096,0,0,0x20000,35
4,2025-05-05 20:38:22.154330,1746448702154330,FUT_TAIFEX_QFF:202505,0,TAIFEX117#12578,923,take,-20,923.5,923.5,...,0,0,0,100,False,0.06846,0,0,0x0,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2025-05-06 13:29:48.152578,1746509388152578,FUT_TAIFEX_QFF:202506,0,TAIFEX117#626654,914,take,-20,915.0,915.0,...,0,0,0,100,False,0.06828,0,0,0x0,19
199,2025-05-06 13:30:25.037869,1746509425037869,FUT_TAIFEX_CDF:202505,0,TAIFEX117#627171,914,alq,1,914.5,914.5,...,0,0,0,2000,False,0.02078,0,0,0x20000,20
200,2025-05-06 13:30:25.038365,1746509425038365,FUT_TAIFEX_QFF:202505,0,TAIFEX117#627243,915,take,-20,915.5,915.5,...,0,0,0,100,False,0.06830,0,0,0x0,21
201,2025-05-06 13:30:37.702568,1746509437702568,FUT_TAIFEX_CDF:202505,0,TAIFEX117#627270,914,alq,1,913.5,913.5,...,0,0,0,2000,True,0.02078,0,0,0x20000,22


In [3]:

# 撈 2025/5/5 的日盤資料
df_day = get_fills('capital_proton_tgf', date=dt.date(2025, 5, 5), night=False)

# 撈夜盤資料

df_day

Key not found: TAIFEX100:20250505


,ts,instr,oflag,tid,px,alqortake,sz,theo,fwd,ffwd,...,gamma,theta,vol,mult,latefill,fee,qm,pfdelta,otags,seq
time,,,,,,,,,,,,,,,,,,,,,
2025-05-05 10:54:18.646587,1746413658646587,FUT_TAIFEX_TGF:202506,0,TAIFEX236#4373,11555,alq,-1,11528.5,11528.5,11528.5,...,0,0,0,100,True,0.128887,0,0,0x20000,1
2025-05-05 12:07:12.345308,1746418032345308,FUT_TAIFEX_TGF:202506,0,TAIFEX236#4477,11613,alq,-2,11610.5,11610.5,11610.5,...,0,0,0,100,False,0.129032,0,0,0x20000,2
2025-05-05 12:07:12.345914,1746418032345914,FUT_TAIFEX_TGF:202506,2097152,TAIFEX236#4479,11615,take,-2,11615.5,11615.5,11615.5,...,0,0,0,100,True,0.129037,0,0,0x60000,3
2025-05-05 12:12:42.811340,1746418362811340,FUT_TAIFEX_TGF:202506,0,TAIFEX236#4498,11608,alq,2,11646.0,11646.0,11646.0,...,0,0,0,100,False,0.129020,0,0,0x20000,4


In [7]:
def compute_pnl_by_instr(df):
    """
    用 FIFO 算法計算每個商品的已實現損益。

    Parameters:
    - df: DataFrame，需包含欄位 'instr', 'px', 'sz', 'mult'
          且 'sz' 為正負，表示方向 + 口數
          index 為時間（已排序）

    Returns:
    - results: dict[instr] -> realized PnL (float)
    - detail_df: DataFrame，包含每筆平倉紀錄（配對資訊與損益）
    """
    df = df.sort_index().copy()
    results = {}
    all_details = []

    for instr, group in df.groupby("instr"):
        group = group.sort_index()
        open_positions = []
        realized_pnls = []

        for _, row in group.iterrows():
            sz = row["sz"]
            px = row["px"]
            mult = row["mult"]
            time = row.name  # index 為時間

            if sz == 0:
                continue

            remaining = sz
            while remaining != 0 and open_positions and (remaining * open_positions[0]["sz"] < 0):
                opp = open_positions[0]
                match_qty = min(abs(remaining), abs(opp["sz"]))
                match_sz = match_qty * np.sign(remaining)

                # 正確計算方向：無論是多→空，或空→多，皆為 sell - buy
                if match_sz > 0:
                    pnl = (opp["px"] - px) * match_qty * mult  # 我是買方
                else:
                    pnl = (px - opp["px"]) * match_qty * mult  # 我是賣方

                realized_pnls.append(pnl)
                all_details.append({
                    "instr": instr,
                    "open_time": opp["time"],
                    "open_px": opp["px"],
                    "close_time": time,
                    "close_px": px,
                    "match_sz": match_sz,
                    "mult": mult,
                    "pnl": pnl,
                })

                remaining -= match_sz
                opp["sz"] += match_sz
                if opp["sz"] == 0:
                    open_positions.pop(0)

            if remaining != 0:
                open_positions.append({
                    "sz": remaining,
                    "px": px,
                    "mult": mult,
                    "time": time,
                })

        results[instr] = sum(realized_pnls)

    detail_df = pd.DataFrame(all_details)
    return results, detail_df

In [11]:

# 假設你已經有 df_day 或 df_all
results, detail_df = compute_pnl_by_instr(df_all)

# 顯示總損益
for instr, pnl in results.items():
    print(f"{instr}: {pnl:,.2f}")

# 顯示前幾筆配對紀錄
detail_df


FUT_TAIFEX_CDF:202503: -468,000.00
FUT_TAIFEX_CDF:202504: -240,000.00
FUT_TAIFEX_CDF:202505: 16,000.00
FUT_TAIFEX_QFF:202503: 682,500.00
FUT_TAIFEX_QFF:202504: 397,500.00
FUT_TAIFEX_QFF:202505: -16,000.00


,instr,open_time,open_px,close_time,close_px,match_sz,mult,pnl
0,FUT_TAIFEX_CDF:202503,0,999,100,989,-1,2000,-20000
1,FUT_TAIFEX_CDF:202503,2,999,179,998,-1,2000,-2000
2,FUT_TAIFEX_CDF:202503,4,999,180,998,-2,2000,-4000
3,FUT_TAIFEX_CDF:202503,7,999,245,1000,-1,2000,2000
4,FUT_TAIFEX_CDF:202503,9,999,323,998,-1,2000,-2000
...,...,...,...,...,...,...,...,...
526,FUT_TAIFEX_QFF:202504,911,967,929,962,-1,100,-500
527,FUT_TAIFEX_QFF:202504,931,964,974,920,1,100,4400
528,FUT_TAIFEX_QFF:202504,931,964,975,919,1,100,4500
529,FUT_TAIFEX_QFF:202505,843,976,853,968,-17,100,-13600


In [10]:
df_all = loader.load_concat("2025-03-01", "2025-03-31")


Loading fills for capital_electron_cdf: 100%|██████████| 31/31 [00:01<00:00, 18.11it/s]
